In [37]:
# Install project dependencies
!pip install torch
!pip install transformers
!pip install sagemaker huggingface

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [29]:
# INIT SAKEMAKER

import sagemaker

# init session to connect to other aws ressource
sess = sagemaker.Session()

# create session bucket to store project artefacts
sagemaker_session_bucket = "sagemaker-bert2bert"
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

# IAM role to allow connection to session bucket and create model deployment
role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::159645953966:role/service-role/SageMaker-MLOps
sagemaker bucket: sagemaker-bert2bert
sagemaker session region: eu-central-1


In [ ]:
# INIT MODEL PIPELINE

from transformers import pipeline, BertTokenizerFast

# Define Model name
model_name = "mrm8488/bert2bert_shared-german-finetuned-summarization"

# Define Tokenizer
tokenizer = BertTokenizerFast.from_pretrained(model_name)

# Init download
summarizer = pipeline("summarization", model=model_name, tokenizer=tokenizer)

In [72]:
# Install bitsandbytes
!pip install git+https://github.com/huggingface/bitsandbytes.git

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Cloning https://github.com/huggingface/bitsandbytes.git to /tmp/pip-req-build-if6rxmxr
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/bitsandbytes.git /tmp/pip-req-build-if6rxmxr
Username for 'https://github.com/huggingface/bitsandbytes.git': ^C
ERROR: Operation cancelled by user


In [73]:
# bitsandbytes Quantization

from transformers import AutoModelForSeq2SeqLM
import bitsandbytes as bnb

model_name = "mrm8488/bert2bert_shared-german-finetuned-summarization"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Ersetze die linearen Schichten durch 8-Bit-quantisierte Versionen
for name, layer in model.named_modules():
    if isinstance(layer, torch.nn.Linear):
        # Ersetze die Schicht durch eine 8-Bit-Version
        setattr(model, name, bnb.optim.GlobalOptimManager.get_optim(layer, optim_bits=8))

# Speichere das quantisierte Modell
model.save_pretrained('./my_quantized_bert2bert_model')
)

# Quantizated model save at model registry (s3)
prefix = 'my-quantized-models/bert2bert'  # Dieser Prefix ist der Pfad in Ihrem Bucket.

# Lokalen Pfad des Modells
model_dir = model_save_directory

# Dateien im Modellverzeichnis auflisten
model_files = os.listdir(model_dir)

# Jede Datei zum S3-Bucket hochladen
for file in model_files:
    local_path = os.path.join(model_dir, file)
    s3_path = f"s3://{bucket}/{prefix}/{file}"
    sagemaker_session.upload_data(path=local_path, bucket=bucket, key_prefix=f"{prefix}/{file}")
    print(f"Uploaded {local_path} to {s3_path}")

SyntaxError: unmatched ')' (2441968150.py, line 17)

In [67]:
# OPTIMZATION: Using BertTokenizerFast

from transformers import pipeline, BertTokenizerFast
#from transformers import pipeline, BertTokenizer
import time

# Define Model name
model_name = "mrm8488/bert2bert_shared-german-finetuned-summarization"

# Define Tokenizer
tokenizer = BertTokenizerFast.from_pretrained(model_name)
#tokenizer = BertTokenizer.from_pretrained(model_name)

# Init download
summarizer = pipeline("summarization", model=model_name, tokenizer=tokenizer)

# start timer
start_time = time.time()

# Test-Input
text_input = (
   "Veränderungen und Innovationen sind allgegenwärtig, eine kontinuierliche und nachhaltige Weiterentwicklung der Mitarbeitenden ist von zentraler Bedeutung. Hier setzt das Praxis-Coaching an, denn es ist eine wirkungsvolle Methode, um individuelle Lern- und Entwicklungsprozesse zu fördern und zu gestalten. Das Praxis-Coaching Data Science unterstützt Sie dabei, den Praxistransfer in den Unternehmenskontext zu optimieren und die Rolle des Data Scientists in Ihrer Arbeitsumgebung zu etablieren. Es hilft Ihnen dabei, an die Themen des Seminars praxisnah anzuknüpfen, und ermöglicht im virtuellen 1:1-Raum mit dem:der Coach:in den Austausch für individuelle Impulse und Tipps. Der Ansatz dieses Coachings beinhaltet weder eine Wissensvermittlung noch eine Unternehmensberatung. Es geht hier in erster Linie darum, Ihre Rolle als Data Scientist gemeinsam mit dem:der Coach:in zu analysieren und Ihr Selbstvertrauen in diesem Bereich zu fördern. Das Praxis-Coaching Data Science adressiert genau dieses Dilemma: Es bringt Ihre Daten-Ideen und Ihr Wissen aus den Fachbereichen mit der Erfahrung unserer Expert:innen bei der erfolgreichen Implementierung von Datenprojekten zusammen. Egal ob eine oder zehn Stunden - buchen Sie über unser Anfrageformular ein flexibel nutzbares Kontingent mit unseren Trainer:innen. Preis pro Stunde: 250€ (297,50€ inkl MwSt)."
    "Geben Sie im nächsten Schritt bei „Anfragen“ im Bemerkungsfeld an, bei welchem:welcher Trainer:in Sie gerne das Coaching durchführen wollen und wie viele Stunden Sie buchen möchten. Der:Die Trainer:in meldet sich daraufhin bei Ihnen."
    "Inhalte"
    "Das Praxis-Coaching Data Science unterstützt Sie bei den folgenden Möglichkeiten:"
    "Entwicklung: Etablierung Ihrer Jobrolle als Data Scientist."
    "Follow-up: Anknüpfung an die Kursinhalte und Reflexion für den maximalen Praxistransfer."
    "Entfaltung: Individuelle Impulse und Tipps für die Umsetzung von Maßnahmen in Ihrem Businesskontext."
    "Ihr Nutzen"
    "Nutzen Sie dieses Praxis-Coaching für die Förderung und Gestaltung Ihres individuellen Lern- und Entwicklungsprozesses:"
    "Schaffen Sie im 1:1 mit dem Experten Klarheit für Ihre neue Rolle als Data Scientist und stärken Sie dieses Mindset für sich und Ihr Team."
)

results = summarizer(text_input)

#print summarizer results
for result in results:
    print(f"Zusammenfassung: {result['summary_text']}")

# end timer
end_time = time.time()

# print time
execution_time = end_time - start_time
print(f"Ausführungszeit: {execution_time} Sekunden")

The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


Zusammenfassung: Veränderungen und Innovationen sind allgegenwärtig, eine kontinuierliche und nachhaltige Weiterentwicklung der Mitarbeitenden ist von zentraler Bedeutung. Hier setzt das Praxis - Coaching an, denn es ist eine wirkungsvolle Methode, um individuelle Lern - und Entwicklungsprozesse zu fördern und zu gestalten.
Ausführungszeit: 4.183176279067993 Sekunden


In [45]:
# Use DLC Container
# https://github.com/aws/deep-learning-containers

# Using own build container
https://github.com/aws/amazon-sagemaker-examples/tree/main/inference/torchserve/mme-gpu/workspace/docker

SyntaxError: invalid syntax (60234713.py, line 5)

In [ ]:
# Use Deepspeed
# https://github.com/aws/amazon-sagemaker-examples/blob/main/inference/generativeai/deepspeed/GPT-J-6B_DJLServing_with_PySDK.ipynb

In [ ]:
# Inference Code from Huggingface transformer libary 
!pygmentize code/inference_code.py


https://github.com/aws-samples/amazon-sagemaker-script-mode/blob/master/deploy-pretrained-model/BERT/Deploy_BERT.ipynb

In [ ]:
# Packaging the model

import tarfile

zipped_model_path = os.path.join(model_path, "model.tar.gz")

with tarfile.open(zipped_model_path, "w:gz") as tar:
    tar.add(model_path)
    tar.add(code_path)

In [ ]:
# Push model package to regitry

In [13]:
# Deploy Model


from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role

endpoint_name = 'bert-base'

model = PyTorchModel(entry_point='inference_code.py', 
                     model_data=zipped_model_path, 
                     role=get_execution_role(), 
                     framework_version='1.5', 
                     py_version='py3')

predictor = model.deploy(initial_instance_count=1, 
                         instance_type='ml.m5.xlarge', 
                         endpoint_name=endpoint_name)

# Was ist der unerschied von model zu huggingface_estimator?
predictor = huggingface_estimator.deploy(1,"ml.g4dn.xlarge")

In [ ]:
predictor.serializer = sagemaker.serializers.JSONSerializer()
predictor.deserializer = sagemaker.deserializers.JSONDeserializer()

In [ ]:
# Inference
# https://docs.aws.amazon.com/de_de/sagemaker/latest/APIReference/API_runtime_InvokeEndpoint.html


import boto3

sm = boto3.client('sagemaker-runtime')

prompt = "The best part of Amazon SageMaker is that it makes machine learning easy."

response = sm.invoke_endpoint(EndpointName=endpoint_name, 
                              Body=prompt.encode(encoding='UTF-8'),
                              ContentType='text/csv')

response['Body'].read()



In [ ]:
# Serverless Inference
from sagemaker.serverless.serverless_inference_config import ServerlessInferenceConfig

serverless_config = ServerlessInferenceConfig(
    memory_size_in_mb=6144,
    max_concurrency=1,
)

# https://github.com/aws/amazon-sagemaker-examples/blob/main/serverless-inference/huggingface-serverless-inference/huggingface-text-classification-serverless-inference.ipynb

In [ ]:
# Add metrics to scale based on traffic
# https://github.com/aws/amazon-sagemaker-examples/tree/main/sagemaker-inference-recommender/auto-scaling


In [ ]:
# HF model registration
# https://github.com/aws/amazon-sagemaker-examples/blob/main/sagemaker-inference-recommender/huggingface-inference-recommender/huggingface-inference-recommender.ipynb


In [14]:
# Clean up
predictor.delete_endpoint()


NameError: name 'predictor' is not defined